In [ ]:
%%bash
pip install astroML
pip install sklearn

In [1]:
import numpy as np
from jointpdf.jointpdf import JointProbabilityMatrix
from jointpdf.jointpdf import FullNestedArrayOfProbabilities

In [2]:
def calculate_entropy(distribution_values):
    """ calculate the entropy
    
    params:
        distribution_values should be a 1-dim numpy array, summing to 1
    
    returns: the entropy value
    """
    dist_values = distribution_values[distribution_values != 0]
    return - np.sum(np.log2(dist_values) * dist_values)

def calulate_mutual_information(distribution1, distribution2, joint_distribution):
    return (calculate_entropy(distribution1) +
            calculate_entropy(distribution2) -
            calculate_entropy(joint_distribution))

In [3]:
from functools import reduce

def compute_joint(multi_index, marginal_distributions):
    return reduce(lambda x, y: x*y, [marginal_distributions[combination] for
            combination in zip(range(len(multi_index)), multi_index)])

def compute_joint_from_marginals(marginal_distributions):
    dimension_joint_distribution = [len(dist) for dist in marginal_distributions]
    full_distribution_temp = np.zeros(dimension_joint_distribution)
    full_distribution = np.zeros(dimension_joint_distribution)

    it = np.nditer(full_distribution, flags=['multi_index'], op_flags=[['readwrite']])
    while not it.finished:
        full_distribution[it.multi_index]= compute_joint(it.multi_index, marginal_distributions)
        it.iternext()

    return full_distribution

marginal_distributions2 = np.array([[0.3, 0.7], [0.3, 0.7]])
print(compute_joint_from_marginals(marginal_distributions2))

def xor(it):
    """apply xor to binary numbers x1 and x2"""
    return 0 if it[0]==it[1] else 1

def _and(it):
    """calculate the and of it
    
    params:
        it: binary iterable of length 2
        
    returns: a binary number 
    """
    return 1 if it[0]==1 and it[1]==1 else 0
    
def subset_and(it, subsets):
    entries = [np.all(np.array(it[subset])==1) for subset in subsets]
    return [1 if entry else 0 for entry in entries]
    
def parity(it):
    """calculate the parity (sum(it)%2) of the iterable
    
    params:
        it: a binary iterable
        
    returns: a binary number 
    """
    
    return sum(it)%2
    
def double_xor(it):
    """apply xor to every pair in it and return the outcome
    
    params:
        it: 1-d iterable with binary entries and len(it)%2==0
    """

    return [xor(it[2*i:2*i+2]) for i in range(len(it)/2)]
        
print(double_xor((1,1,0,1)))




[[ 0.09  0.21]
 [ 0.21  0.49]]
[0, 1]


In [65]:
class JointProbabilityMatrixExtended(JointProbabilityMatrix):
    """this class extends the JointProbabilityMatrix by allowing different
    variables to have different statespaces"""
    def __init__(self, numvariables, max_num_states, joint_pdf):
        self.numvariables = numvariables
        self.max_num_states = max_num_states
        self.joint_probabilities = joint_pdf 
        #super(JointProbabilityMatrix, self).__init__(num_variables, num_states, joint_pdf)
    
    #needed to access methods in parent class
    def adjust_to_old_format(self):
        """ensure that every variable in joint has as many states and add states as needed"""
        self.old_joint_probabilities = self.joint_probabilities
        max_number_of_states = max(self.joint_probabilities.joint_probabilities.shape)
        temp_joint_probabilities = np.zeros([max_number_of_states]*self.numvariables)
        it = np.nditer(self.joint_probabilities.joint_probabilities, flags=['multi_index'])
        while not it.finished:
            temp_joint_probabilities[it.multi_index] = it.value
            it.iternext()
            
        self.joint_probabilities.joint_probabilities = temp_joint_probabilities
        self.numvalues = max_number_of_states
    
    #rewrite this to work on a nd-array instead!
    def append_determinstic_function(self, func, num_variables, num_states):
        """append an extra variable to the distribution given by the 
        deterministic function func
        
        params:
            func: a deterministic discrete function over the already defined variables
            num_variables: the number of variables
            num_states: iterable, the number of outcomes of the function
        """
        old_shape = list(self.joint_probabilities.joint_probabilities.shape)
        new_shape = old_shape + num_states
        dummy_joint_probability = np.zeros(new_shape)
        temp2_joint_probability = np.zeros(new_shape)
            
        it = np.nditer(dummy_joint_probability, flags=['multi_index'])
        while not it.finished:
            arguments = tuple(list(it.multi_index)[:-num_variables])
            if np.all(np.array(func(arguments)) == np.array(it.multi_index[-num_variables:])):
                temp2_joint_probability[it.multi_index] = self.joint_probabilities.joint_probabilities[arguments]
                
            it.iternext()
            
        self.joint_probabilities.joint_probabilities = temp2_joint_probability
        self.numvariables = self.numvariables + num_variables 
                
    

First define some marginal distribution that are assumed to be independent 

In [66]:
marginal_distributions = np.array([[0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5,  0.5]])

Tryout whether the implementation works

In [67]:
pdf = JointProbabilityMatrix(4, 2, compute_joint_from_marginals(marginal_distributions))
pdf_extended = JointProbabilityMatrixExtended(
    4, 2, FullNestedArrayOfProbabilities(compute_joint_from_marginals(marginal_distributions))
)
pdf_extended.adjust_to_old_format()

marginal_extended = pdf_extended.marginalize_distribution([0, 1])
marginal = pdf.marginalize_distribution([0, 1])

pdf_extended.append_determinstic_function(double_xor, num_variables=2, num_states=[2, 2])
pdf_extended.adjust_to_old_format()
marginal_extended = pdf_extended.marginalize_distribution([0, 1])

print(
    np.all(
        marginal_extended.joint_probabilities.joint_probabilities==marginal.joint_probabilities.joint_probabilities
    )
)


True


In [68]:
import itertools

def calculate_mi_profile(joint_distribution, function_labels):
    """create a mutual information profile
    
    params:
        joint_distribution: A JointProbabilityMatrix object from the jointpdf package.
        The joint probabilities properties should be of both all the variables and the
        function.
        function_labels: The labels in the joint distribution which the function represents
        
    returns: a 1-d nd-array where every value represents the average normalized mutual information
        between subsets of variables (of the size of the index) and the ouput variable. 
    """

    number_of_arguments = joint_distribution.numvariables-len(function_labels)
    mi_values = np.zeros(number_of_arguments+1)
    for number_of_variables in np.arange(1, number_of_arguments+1, 1):
        combinations = itertools.combinations(range(number_of_arguments), 
                                              number_of_variables)
        
        mi_values[number_of_variables] = np.mean(
            [joint_distribution.mutual_information(list(combination), function_labels) 
             for combination in combinations]
        )

        for comb in itertools.combinations(range(number_of_arguments), number_of_variables):
            print("input variables {} function labels {} mi value {}".format(
                list(comb), 
                function_labels,
                joint_distribution.mutual_information(list(comb), function_labels)
            ))
        
    return mi_values
                
    

Do some testing. I found out that xor for i.i.d variables has mutual information with the individual variables.
This was a huge surprise so I tested it with multiple implementation and also using playground.ipnb. 

In [69]:
pdf_extended = JointProbabilityMatrixExtended(
    4, 2, FullNestedArrayOfProbabilities(compute_joint_from_marginals(marginal_distributions))
)

pdf_extended.append_determinstic_function(double_xor, num_variables=2, num_states=[2, 2])
pdf_extended.adjust_to_old_format()
print(calculate_mi_profile(pdf_extended, [4, 5]))

print("now using the create function")
print("")

pdf_extended2 = JointProbabilityMatrixExtended(
    4, 2, FullNestedArrayOfProbabilities(compute_joint_from_marginals(marginal_distributions))
)

def create_mutual_information_profile(pdf, function, dim_output_function, num_states_output_function):
    """create mutual information profile values
    
    params:
        pdf: An object of the JointProbabilityMatrixExtendedClass
        function: a function object that takes an iterable of the same length
        as the number of variables in the pdf_extended object
        dim_output_function: the dimension (number of variables) the function outputs
        num_states_output_function: a list, where every entry represents the amount of 
        states that variable has. The length should be equal to dim_output_function
        
    returns: An array representing the MI profile values for number of variables 
    starting at zero and going to the total amount of variables in the distribution
    """
     
    
    pdf.append_determinstic_function(double_xor, dim_output_function, num_states_output_function)
    pdf.adjust_to_old_format()
    
    function_indices = list(range(pdf.numvariables-dim_output_function, pdf.numvariables))
    
    print("the function indices are {}".format(function_indices))
    return calculate_mi_profile(pdf_extended, function_indices)

print(create_mutual_information_profile(pdf_extended2, double_xor, 2, [2,2]))

input variables [0] function labels [4, 5] mi value 0.0
input variables [1] function labels [4, 5] mi value 0.0
input variables [2] function labels [4, 5] mi value 0.0
input variables [3] function labels [4, 5] mi value 0.0
input variables [0, 1] function labels [4, 5] mi value 1.0
input variables [0, 2] function labels [4, 5] mi value 0.0
input variables [0, 3] function labels [4, 5] mi value 0.0
input variables [1, 2] function labels [4, 5] mi value 0.0
input variables [1, 3] function labels [4, 5] mi value 0.0
input variables [2, 3] function labels [4, 5] mi value 1.0
input variables [0, 1, 2] function labels [4, 5] mi value 1.0
input variables [0, 1, 3] function labels [4, 5] mi value 1.0
input variables [0, 2, 3] function labels [4, 5] mi value 1.0
input variables [1, 2, 3] function labels [4, 5] mi value 1.0
input variables [0, 1, 2, 3] function labels [4, 5] mi value 2.0
[ 0.          0.          0.33333333  1.          2.        ]
now using the create function

the function ind

Just created the create_mutual_information_profile to automate some steps when creating an MI profile

In [56]:

#so to calculate the mutual information profile of a function
#take the following steps

#create a new pdf holding the joint pdf of the variables
pdf_extended = JointProbabilityMatrixExtended(
    2, 2, FullNestedArrayOfProbabilities(compute_joint_from_marginals(marginal_distributions[:2]))
)

    
#state the amount of variables the function outputs
#and the number of states of each variable

#xor_output_dim = 1
#xor_num_states = [2]

#print("for xor the mi profile is {}".format(
#    create_mutual_information_profile(pdf_extended, xor, xor_output_dim, xor_num_states)
#))

#create a new pdf holding the joint pdf of the variables
pdf_extended = JointProbabilityMatrixExtended(
    4, 2, FullNestedArrayOfProbabilities(compute_joint_from_marginals(marginal_distributions))
)
    
#state the amount of variables the function outputs
#and the number of states of each variable

double_xor_output_dim = 2
double_xor_num_states = [2, 2]

print("for xor the mi profile is {}".format(
    create_mutual_information_profile(pdf_extended, double_xor, double_xor_output_dim, double_xor_num_states)
))

pdf_extended = JointProbabilityMatrixExtended(
    4, 2, FullNestedArrayOfProbabilities(compute_joint_from_marginals(marginal_distributions))
)
pdf_extended.append_determinstic_function(double_xor, 
                                              num_variables=2, 
                                              num_states=[2, 2])

pdf_extended.adjust_to_old_format()

print(pdf_extended.mutual_information([0,1,2,3], [4, 5]))


6
[[[[[[ 0.0625  0.    ]
     [ 0.      0.    ]]

    [[ 0.0625  0.    ]
     [ 0.      0.    ]]]


   [[[ 0.0625  0.    ]
     [ 0.      0.    ]]

    [[ 0.0625  0.    ]
     [ 0.      0.    ]]]]



  [[[[ 0.      0.    ]
     [ 0.      0.0625]]

    [[ 0.      0.    ]
     [ 0.      0.0625]]]


   [[[ 0.      0.    ]
     [ 0.      0.0625]]

    [[ 0.      0.    ]
     [ 0.      0.0625]]]]]




 [[[[[ 0.      0.    ]
     [ 0.      0.0625]]

    [[ 0.      0.    ]
     [ 0.      0.0625]]]


   [[[ 0.      0.    ]
     [ 0.      0.0625]]

    [[ 0.      0.    ]
     [ 0.      0.0625]]]]



  [[[[ 0.0625  0.    ]
     [ 0.      0.    ]]

    [[ 0.0625  0.    ]
     [ 0.      0.    ]]]


   [[[ 0.0625  0.    ]
     [ 0.      0.    ]]

    [[ 0.0625  0.    ]
     [ 0.      0.    ]]]]]]
the function indices are:
[4, 5]


IndexError: tuple index out of range

In [ ]:
pdf_extended = JointProbabilityMatrixExtended(
    2, 2, FullNestedArrayOfProbabilities(compute_joint_from_marginals(marginal_distributions))
)

#print(pdf_extended.joint_probabilities.joint_probabilities)
#print("check mutual info {}".format(pdf_extended.mutual_information([0], [1])))

pdf_extended.append_determinstic_function(double_xor, num_variables=2, num_states=[2, 2])
pdf_extended.adjust_to_old_format()

#print(pdf_extended.joint_probabilities.joint_probabilities)
#print("second check mutual information {}".format(pdf_extended.mutual_information([0], [2])))

#calulate_mutual_information(
#    pdf_extended.marginalize_distribution([0]).joint_probabilities.joint_probabilities,
#    pdf_extended.marginalize_distribution([2]).joint_probabilities.joint_probabilities,
#    pdf_extended.marginalize_distribution([0,2]).joint_probabilities.joint_probabilities
#)

calculate_mi_profile(pdf_extended, [4, 5])